In [1]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
#import numpy as np
from numpy.linalg import norm
import psycopg2
import clingo
from clingo.control import Control
import json
from yaml import load, Loader
from helper import *
from stories import *
#import multiprocessing

In [2]:
canonical_dict = {"sudoku":os.path.join("projects","sudoku","canonical_sudoku.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","canonical_seeknumbers2.lp"),
                 "minotaur":os.path.join("projects","minotaur","canonical_minotaur1.lp"),
                 "creek":os.path.join("projects","creek","canonical_creek.lp"),                 
                 "yosenabe":os.path.join("projects","yosenabe","canonical_yosenabe.lp"),
                 "hop":os.path.join("projects","hop","canonical_hop.lp"),
                 "lights":os.path.join("projects","lights","canonical_lights.lp")}

In [3]:
instances_dict ={"sudoku":os.path.join("projects","sudoku","instances","ex01.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","instances","ex01.lp"),
                 "minotaur":os.path.join("projects","minotaur","instances","level01.lp"),
                 "creek":os.path.join("projects","creek","instances","ex01.lp"),
                 "yosenabe":os.path.join("projects","yosenabe","instances","instance01.lp"),
                 "hop":os.path.join("projects","hop","instances","level1.lp"),
                 "lights":os.path.join("projects","lights","instances","test01.lp")}

In [4]:
solution_dict ={"sudoku":os.path.join("projects","sudoku","solutions","ex01.json"),
                 "seeknumbers":os.path.join("projects","seeknumbers","solutions","ex01.json"),
                 "minotaur":os.path.join("projects","minotaur","solutions","level01.json"),
                 "creek":os.path.join("projects","creek","solutions","ex01.json"),
                 "yosenabe":os.path.join("projects","yosenabe","solutions","solution01.txt"),
                 "hop":os.path.join("projects","hop","solutions","solution01.txt"),
                 "lights":os.path.join("projects","lights","solutions","solution01.txt")}

In [5]:
def asp_try_5(asp_file,instance):
    models = []  
    errors = []
    symbols = []
    messages = []

    def custom_logger(code, message):
        messages.append(message)
        
    def on_model(model):
        #print("Model:", model)
        models.append(model)
        with open('output.txt','w') as file:
            print(model, file=file)
        with open('output.txt','r') as file:
            lines = file.readlines()
            symbols_list = lines[0].split()
            #for s in symbols_list:
            symbols.append(symbols_list)
        os.remove("output.txt")

    time_init = time.time() 
    asp_program = []
    # TODO: timeout of Unix systems. Wrapper around. Kill the process. 
    control = clingo.Control(['--warn=none','-t','5','-n','10'], logger=custom_logger) #, '--warn=none', '--opt-mode=optN' --time-limit=5, -t 5
    input_files = [asp_file, instance]
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
            
    try:
        control.add("base", [], "".join(asp_program))
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    try:
        control.ground([("base", [])])
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    #control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    time_final = time.time() 
    delta_time = time_final-time_init
    
    if delta_time > 5 and len(symbols) == 0:
        errors.append("Timeout")

    #print(f"Results: {models}, {errors}, {symbols}. Timedelta {delta_time}")
    return models, errors, symbols, messages

In [6]:
def get_symbols_from_solution(solution_path):        
    with open(solution_path,'r') as file:
        if ".json" in solution_path:
            data = json.load(file)
            results = data["Call"][0]["Witnesses"] #[0]["Value"]
            #print(result)
        else:
            lines = file.readlines()
            splitted = lines[0].split()
            results = [{"Value":splitted}]
    return results

In [7]:
def jaccard_similarity(lista1, lista2):
    set1 = set(lista1)
    set2 = set(lista2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return instersection/union

In [8]:
def get_metrics(symbols, solutions):
    symbols_set = set(symbols)
    matchs = 0
    final_tp = set()
    final_fp = ()
    final_fn = ()
    for solution in solutions:
        solution_set = set(solution["Value"])
        union = len(symbols_set.union(solution_set))
        tp = solution_set.intersection(symbols_set)
        matchs_temp = len(tp)
        solution_len = len(solution_set)
        if matchs_temp >= matchs:            
            matchs = matchs_temp
            final_tp = tp
            final_fp = symbols_set.difference(solution_set)
            final_fn = solution_set.difference(symbols_set)
            solution_len = len(solution_set)
    accuracy = matchs / solution_len if solution_len != 0 else 0 
    precision = matchs / (len(final_tp) + len(final_fp)) if (len(final_tp) + len(final_fp)) != 0 else 0  
    recall = matchs / (len(final_tp) + len(final_fn)) if (len(final_tp) + len(final_fn)) != 0 else 0 
    f1_score = 2 * ((precision * recall) / (precision + recall)) if (precision + recall) != 0 else 0
    jaccard = matchs / union if union != 0 else 0
    #print(f"Accuracy: {accuracy:.4f}. Precision: {precision:.4f}. Recall: {recall:.4f}. F1: {f1_score:.4f}")
    return matchs, round(accuracy,2), round(precision,2), round(recall,2), round(f1_score,2), round(jaccard,2)

In [9]:
def get_best_metrics(all_symbols,solutions):
    matchs = acc = prec = rec = f1 = jac = 0
    for symbols in all_symbols:
        matchs_t, acc_t, prec_t, rec_t, f1_t, jac_t = get_metrics(symbols, solutions)
        if matchs_t > matchs:
            matchs = matchs_t
            acc = acc_t
            prec = prec_t
            rec = rec_t
            f1 = f1_t
            jac = jac_t
    return matchs, acc, prec, rec, f1, jac

In [10]:
def repetition_counter(file):
    with open(file,'r') as f:
        lines = f.readlines()
        lines = [x for x in lines if x!='\n']
        #print(f"Lines {lines}")
        list_unique_lines = list(set(lines))
        unique_lines = len(list_unique_lines)
        #print(f"List Unique Lines: {list_unique_lines}")
        return len(lines) - unique_lines

In [11]:
def evaluate(file, instance, solution_path, canonical_file):
    models, errors, symbols, messages = asp_try_5(file,instance)
    n_errors = len(errors)
    errors = "".join(errors)
    n_models = len(models)    
    canonical_lines = lines_counter(canonical_file)
    encoding_lines = lines_counter(file)
    repeated_lines = repetition_counter(file)
    #print("Length of Canonical:" + str(canonical_lines) + ". Length Encoding: "+ str(encoding_lines))    
    diff_lines = abs(canonical_lines - encoding_lines)
    solutions = get_symbols_from_solution(solution_path)
    #print("Symbols: " + str(symbols))
    #print("Solution: " + str(solution))
    matchs, accuracy, precision, recall, f1_score, jaccard = get_best_metrics(symbols, solutions) 
    at_least_a_model = 1 if n_models > 0 else 0
    kpi = 25 + at_least_a_model*25 + 10*(accuracy + precision + recall + f1_score + jaccard) - 2*(diff_lines + repeated_lines + n_errors)
    result_dict = {"kpi":kpi, "diff_lines":diff_lines, "repeated":repeated_lines, "n_errors":n_errors, "n_models":n_models, "errors":errors, "symbols":symbols, "messages":messages, "matchs":matchs,
                    "accuracy":accuracy, "precision":precision, "recall":recall, "f1_score":f1_score, "jaccard":jaccard}
    #print(f"Precision: {precision}")
    print_out_dict = {x: result_dict[x] for x in result_dict if x not in ["symbols"]}
    print(print_out_dict)
    return result_dict

In [12]:
!clingo creek2.lp projects/creek/instances/ex01.lp

clingo version 5.6.2
Reading from creek2.lp ...
Solving...
UNSATISFIABLE

Models       : 0
Calls        : 1
Time         : 0.006s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.016s


In [13]:
results = evaluate('creek1.lp',instances_dict['creek'],solution_dict['creek'],canonical_dict['creek'])

{'kpi': 9, 'diff_lines': 7, 'repeated': 0, 'n_errors': 1, 'n_models': 0, 'errors': 'parsing failed', 'messages': ['<block>:33:108-109: error: syntax error, unexpected =, expecting )\n'], 'matchs': 0, 'accuracy': 0, 'precision': 0, 'recall': 0, 'f1_score': 0, 'jaccard': 0}


In [14]:
conn_string = "dbname=thesis user=postgres password=postgres"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


In [ ]:
sql = "CREATE TABLE IF NOT EXISTS newtry AS (SELECT * FROM panel)"
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
cursor.close()

In [15]:
def update_database(conn, file, kpi, diff_lines, repeated, n_errors, n_models, errors, symbols, messages, matchs, accuracy, precision, recall, f1_score, jaccard):
    cursor = conn.cursor()
    symbols = str(symbols).replace("'","")
    command = '''INSERT INTO newtry (file, kpi, diff_lines, repeated, n_errors, n_models, errors, symbols, messages, matchs, accuracy, precision, recall, f1_score, jaccard) 
                VALUES ('%(file)s', '%(kpi)s', '%(diff_lines)s', '%(repeated)s', '%(n_errors)s', '%(n_models)s', '%(errors)s', '%(symbols)s', '%(messages)s', '%(matchs)s', '%(accuracy)s', '%(precision)s', '%(recall)s', '%(f1_score)s',
                 '%(jaccard)s');''' % {"file":file, "kpi": kpi, "diff_lines": diff_lines, "repeated":repeated, "n_errors": n_errors, "n_models": n_models, "errors":errors, "symbols": symbols, "messages": messages, "matchs":matchs, "accuracy":accuracy, "precision":precision, "recall":recall, "f1_score":f1_score, "jaccard":jaccard}
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
        print(f"Insertion of results of file {file} successful")
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=} Result not inserted.")
        conn.commit()
        cursor.close()
        pass

In [16]:
def look_file(conn, file):
    cursor = conn.cursor()
    command = "SELECT * FROM results WHERE file = '"+ file + "'"
    results = []
    try:
        cursor.execute(command)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=} Bad search.")
        conn.commit()
        cursor.close()
        pass
    return results

## Completion of Failed Files

In [18]:
def failed_files_results(conn):
    cursor = conn.cursor()
    command = "SELECT file, story, encoding, messages FROM panel WHERE version IN (9,10) AND messages <> '[]'"
    results = []
    try:
        cursor.execute(command)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=} Bad search.")
        conn.commit()
        cursor.close()
        pass
    return results

In [19]:
failed_results = failed_files_results(conn)

In [20]:
projects = get_useful_projects()

In [21]:
problem_dict = {}
lines_dict = {}
for project in projects:
    problem_dict[project['story']] = project['problem'] 
    lines_dict[project['story']] = project['lines'] 

In [23]:
selected_files = [x[0] for x in failed_results]
selected_stories = [x[1] for x in failed_results]
#selected_encoding = [x[2] for x in failed_results]
selected_encoding = []
for key in lines_dict:
    selected_encoding.append(lines_dict[key])
selected_messages = [x[3] for x in failed_results]

In [24]:
len(selected_files)

521

In [25]:
with open("config.yml","r") as config:
    data = load(config, Loader=Loader)
    api_key = data["apikey"]
    openai.api_key  = api_key

In [ ]:
def try_again(story,encoding,message,printable=False):
    prompt=f'''Given the description of a problem, an logic program in Answer Set Programming and the error messages 
                of the errors of that program, fix the logic program. Do not explain the new solution, but just generate the code.
    
    Problem:
    {story}
    
    Logic Program:
    {encoding}

    Messages:
    {message}
    
    Fixed program:'''
    results = get_completion(prompt)
    if printable: 
        print(f"Prompt: {prompt}")
        print(f"Results: {results}")
        print("=="*50)
    return results

In [ ]:
for file, story, encoding, message in zip(selected_files,selected_stories,selected_encoding,selected_messages):
    result = try_again(problem_dict[story],encoding,message)
    update_in_database(file, result)
        